In [ ]:
import numpy as np
import imageio.v3 as imageio
import matplotlib.pyplot as plt
from collections import deque
import uuid

In [ ]:
def is_valid_pixel(universe: np.ndarray, x: int, y: int) -> bool:
    """Check if the pixel is within bounds and is black (0)."""
    return 0 <= x < universe.shape[1] and 0 <= y < universe.shape[0] and universe[y, x] == 0

In [ ]:
def get_neighbors(x: int, y: int, direction_preference: str = None) -> list[tuple[int, int]]:
    """Return 4-connected neighbors, optionally prioritizing directions."""
    neighbors = [(x, y-1), (x, y+1), (x-1, y), (x+1, y)]  # up, down, left, right
    if direction_preference == "right":
        # Prioritize right, down, left, up
        return [(x+1, y), (x, y+1), (x-1, y), (x, y-1)]
    elif direction_preference == "left":
        # Prioritize left, up, right, down
        return [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]
    return neighbors

In [ ]:
def path_exists(universe: np.ndarray, start_x: int, start_y: int, end_x: int, end_y: int,
                direction_preference: str = None) -> tuple[bool, list[tuple[int, int]]]:
    """Find if a path exists between start and end points using BFS. Return (exists, path)."""
    if not (is_valid_pixel(universe, start_x, start_y) and is_valid_pixel(universe, end_x, end_y)):
        return False, []

    queue = deque([(start_x, start_y)])
    visited = {(start_x, start_y)}
    parent = {(start_x, start_y): None}

    while queue:
        x, y = queue.popleft()
        if (x, y) == (end_x, end_y):
            # Reconstruct path
            path = []
            current = (x, y)
            while current is not None:
                path.append(current)
                current = parent[current]
            return True, path[::-1]  # Reverse to get start to end

        for nx, ny in get_neighbors(x, y, direction_preference):
            if is_valid_pixel(universe, nx, ny) and (nx, ny) not in visited:
                visited.add((nx, ny))
                parent[(nx, ny)] = (x, y)
                queue.append((nx, ny))

    return False, []

In [ ]:
def visualize_path(universe: np.ndarray, path: list[tuple[int, int]], output_file: str):
    """Visualize a single path on the universe and save to output_file."""
    vis_image = np.stack([universe, universe, universe], axis=-1)  # Grayscale to RGB
    for x, y in path:
        vis_image[y, x] = [255, 0, 0]  # Mark path in red

    plt.imshow(vis_image)
    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
def two_non_intersecting_paths(
    universe: np.ndarray,
    start1_x: int, start1_y: int, end1_x: int, end1_y: int,
    start2_x: int, start2_y: int, end2_x: int, end2_y: int
) -> tuple[bool, list[tuple[int, int]], list[tuple[int, int]]]:
    """Find two non-intersecting paths, trying right and left preferences for the second path."""
    # Find first path (no direction preference, shortest path)
    exists1, path1 = path_exists(universe, start1_x, start1_y, end1_x, end1_y)
    if not exists1:
        return False, [], []

    # Create a temporary universe blocking the first path
    temp_universe = universe.copy()
    for x, y in path1:
        temp_universe[y, x] = 255  # Mark path1 pixels as white (blocked)

    # Try second path with "right" preference
    exists2, path2 = path_exists(temp_universe, start2_x, start2_y, end2_x, end2_y, direction_preference="right")
    if exists2:
        return True, path1, path2

    # If right fails, try second path with "left" preference
    exists2, path2 = path_exists(temp_universe, start2_x, start2_y, end2_x, end2_y, direction_preference="left")
    if exists2:
        return True, path1, path2

    return False, [], []

In [ ]:
def visualize_two_paths(universe: np.ndarray, path1: list[tuple[int, int]], path2: list[tuple[int, int]], output_file: str):
    """Visualize two non-intersecting paths on the universe and save to output_file."""
    vis_image = np.stack([universe, universe, universe], axis=-1)
    for x, y in path1:
        vis_image[y, x] = [255, 0, 0]  # First path in red
    for x, y in path2:
        vis_image[y, x] = [0, 255, 0]  # Second path in green

    plt.imshow(vis_image)
    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()

In [30]:
# Stimulation
if __name__ == "__main__":
    # Load image
    try:
        universe = imageio.imread('bars.png', mode='L')  # Load as grayscale
    except FileNotFoundError:
        print("Please provide a valid image file (e.g., bars.png).")
        exit(1)

    # Single path
    start_x, start_y = 0, 1
    end_x, end_y = 0, 8
    exists, path = path_exists(universe, start_x, start_y, end_x, end_y)
    if exists:
        print("Single path exists!")
        visualize_path(universe, path, "path_output.png")
    else:
        print("No single path exists.")

    # Two non-intersecting paths
    start2_x, start2_y = 0,0
    end2_x, end2_y = 2, 2
    exists, path1, path2 = two_non_intersecting_paths(
        universe, start_x, start_y, end_x, end_y, start2_x, start2_y, end2_x, end2_y
    )
    if exists:
        print("Two non-intersecting paths exist!")
        visualize_two_paths(universe, path1, path2, "two_paths_output.png")
    else:
        print("No two non-intersecting paths exist.")

Single path exists!
Two non-intersecting paths exist!
